In [4]:
import pandas as pd
import numpy as np
import math
import datetime
import tqdm as tqdm

In [6]:
cases = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_cases.csv', index_col=0)

cases.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3/customer_cases.csv', index=False, encoding='utf-8', sep=',')
print(len(cases))
print(cases['reason'].unique())
print(cases['channel'].unique())
cases.sort_values(by='date_time')

330512
['signup' 'support']
['phone' 'email']


,case_id,date_time,customer_id,channel,reason
1,CC101,2017-01-01 10:32:03,C2448,phone,signup
2,CC102,2017-01-01 11:35:47,C2449,phone,signup
3,CC103,2017-01-01 11:37:09,C2450,phone,signup
4,CC104,2017-01-01 13:28:14,C2451,phone,signup
5,CC105,2017-01-01 13:52:22,C2452,phone,signup
...,...,...,...,...,...
330508,CC330608,2021-12-31 19:16:28,C173079,email,support
330509,CC330609,2021-12-31 19:36:42,C497581,email,support
330510,CC330610,2021-12-31 20:30:02,C127630,email,support
330511,CC330611,2021-12-31 21:30:00,C497538,email,support


In [164]:
dim_geo = pd.read_excel(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\dim_geo.xlsx')
dim_geo.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_geo.csv', sep=',', encoding='utf-8', index=False)
dim_geo.head()

,geo_id,Kommune,Region
0,1,Albertslund,Region Hovedstaden
1,2,Allerød,Region Hovedstaden
2,3,Ballerup,Region Hovedstaden
3,4,Bornholm,Region Hovedstaden
4,5,Brøndby,Region Hovedstaden


In [33]:
dim_customer = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_info.csv', index_col=0)

# Add age group
groups = [10,20,30,40,50,60,70,80,90]
dim_customer['age_group'] = pd.cut(dim_customer['age'], bins=groups, right=True).astype(str).str.replace(', ','-', regex=True)

# Add geography
dim_customer['geo_id'] = pd.Series(np.random.randint(1, 98, size=len(dim_customer))).astype('Int64')


dim_customer.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_customer.csv', sep=',', index=False, encoding='utf-8', )
print(len(dim_customer))
dim_customer.head()

508932


,customer_id,age,gender,age_group,geo_id
1,C2448,76,female,(70-80],72
2,C2449,61,male,(60-70],77
3,C2450,58,female,(50-60],51
4,C2451,62,female,(60-70],91
5,C2452,71,male,(70-80],70


In [34]:
product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_product.csv', index_col=0)
product['cancel_date_time'] = product['cancel_date_time'].replace(np.nan, '1/1/2100').astype(np.datetime64)
product['signup_date_time'] = product['signup_date_time'].astype(np.datetime64)
product['signup_month'] = product['signup_date_time'].dt.to_period('M')
product['cancel_month'] = product['cancel_date_time'].dt.to_period('M')



product.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_product.csv', sep=',', index=False, encoding='utf-8')
print(len(product))
product.head()

508932


,customer_id,product,signup_date_time,cancel_date_time,signup_month,cancel_month
1,C2448,prd_1,2017-01-01 10:35:09,2100-01-01 00:00:00,2017-01,2100-01
2,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,2017-01,2021-09
3,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,2017-01,2019-01
4,C2451,prd_2,2017-01-01 13:32:08,2100-01-01 00:00:00,2017-01,2100-01
5,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,2017-01,2021-06


In [35]:
def num_active_user_in_a_period(start_date):
    '''count active users'''
    number_of_active =  sum((product['signup_month'] <= start_date) & (product['cancel_month'] >= start_date))
    return number_of_active


def who_is_active_in_period(start_date):
    '''count active users'''
    active_users =  (product['signup_month'] <= start_date) & (product['cancel_month'] >= start_date)
    return product[active_users]['customer_id'].to_numpy()


def who_exists(start_date):
    '''count active users'''
    active_users =  (product['signup_month'] <= start_date) 
    return product[active_users]['customer_id'].reset_index(drop=True).to_numpy()


In [51]:
fact_active = pd.DataFrame()
date_list = pd.date_range(start='1/1/2017', end='1/1/2022').to_period('M').unique().to_numpy()

# List of customers who signed up and caceled by month
signup = product.groupby(['signup_month'])['customer_id'].apply(list)
cancel = product.groupby(['cancel_month'])['customer_id'].apply(list)   


# Generate fact table
for d in tqdm.tqdm(date_list[:-1]):
    
    # Get users who have been created at time d
    _users =  who_exists(d)

    # store date d, as np.array
    dates = np.repeat(d, len(_users))

    # Zip/Stack dates and users
    _df = pd.DataFrame(np.dstack((dates,_users))[0], columns = ['date','customer_id']).set_index('date')

    # is_active column
    _df['active'] = _df['customer_id'].isin(who_is_active_in_period(d))

    # new customer this month
    _df['new_customer'] = _df['customer_id'].isin(signup[d])

    # custumers who canceled
    _df['cancel'] = _df['customer_id'].isin(cancel[d])

    # Append these users 
    fact_active = pd.concat([fact_active,_df], ignore_index=False)


fact_active.index = fact_active.index.strftime('%Y-%m-%d')
print(fact_active)
fact_active.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active.csv', encoding='utf-8', sep=',', index=True)

100%|██████████| 60/60 [00:11<00:00,  5.13it/s]


           customer_id  active  new_customer  cancel
date                                                
2017-01-31       C2448    True          True   False
2017-01-31       C2449    True          True   False
2017-01-31       C2450    True          True   False
2017-01-31       C2451    True          True   False
2017-01-31       C2452    True          True   False
...                ...     ...           ...     ...
2021-12-31     C511375    True          True   False
2021-12-31     C511376    True          True   False
2021-12-31     C511377    True          True   False
2021-12-31     C511378    True          True   False
2021-12-31     C511379    True          True   False

[11683870 rows x 4 columns]


In [9]:
dim_product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\product_info.csv', index_col=0)
dim_product.head()

,name,price,billing_cycle
product_id,,,
prd_1,annual_subscription,1200,12
prd_2,monthly_subscription,125,1
